# He I Diagnostics Using FIRS Data

## 2. Single pixel inversion
Shuo Wang

Dept. of Astronomy, NMSU

DKIST Ambassador

### Load Data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import readsav
import hazel
import h5py

In [ ]:
le,r = 100,172 
s4 = np.load('clean.npy')[:,:,le:r]
print(s4.shape)

In [ ]:
wva = readsav('wva.sav')['wva'][le:r]
print(f'{wva[0]:.3f}',f'{wva[-1]:.3f}',wva.shape[0])

 ### Select Line Profiles

In [ ]:
y1,x1 = 197,244
y2,x2 = 140,311
plt.figure(figsize=(12,8))
plt.plot(wva,s4[0,y1,:,x1]/np.max(s4[0,y1,:,x1]),label = 'px1')
plt.plot(wva,s4[0,y2,:,x2]/np.max(s4[0,y2,:,x2]),label = 'px2')
plt.axvline(x=10830.3,color='C3')
plt.legend()

### Prepare Input Files for HAZEL

In [ ]:
stokes = s4[:,y1,:,x1]
stokes[0]/=np.max(stokes[0])
_,nw = stokes.shape
plt.figure(figsize=(12,8))
for i in range(4):
    plt.subplot(221+i)
    plt.plot(wva,stokes[i])

sigma = np.array([[1e-2,5e-4,5e-4,5e-4],]*nw, dtype=np.float64) # noise IQUV
f = open('x244y197.1d', 'wb')
f.write(b'# LOS theta_LOS, phi_LOS, gamma_LOS\n')
f.write(b'0.0 0.0 90.0\n')
f.write(b'\n')
f.write(b'# Boundary condition I/Ic(mu=1), Q/Ic(mu=1), U/Ic(mu=1), V/Ic(mu=1)\n')
f.write(b'1.0 0.0 0.0 0.0\n')
f.write(b'\n')
f.write(b'# SI SQ SU SV sigmaI sigmaQ sigmaU sigmaV\n')
tmp = np.hstack([stokes.T, sigma])
np.savetxt(f, tmp)
f.close()

### Run HAZEL Inversion

In [ ]:
mod = hazel.Model('conf.ini', working_mode='inversion')
mod.read_observation()
mod.open_output()
mod.invert()
mod.write_output()
mod.close_output()

### Results

In [ ]:
fo = h5py.File('output.h5', 'r')
ch1 = fo['ch1']
arr = np.array(['deltav','tau','v'])
for i in arr:
    print(i,': ',f'{ch1[i][0,0,0]:.2f}')
iq=['I','Q','U','V']
plt.figure(figsize = (12,8))
for i in range(4):
    plt.subplot(221+i)
    plt.plot(wva, stokes[i],'.',label='observation')
    plt.plot(wva, fo['spec1']['stokes'][0,0,i],label='inversion')
    plt.xlabel('Wavelength [$\AA$]')
    plt.ylabel(iq[i])
plt.legend()
fo.close()